# 02. Pretrained Models

In this lab, you will learn how to leverage pre-trained models to build image classifiers instead of building a model from scratch.


## Import Libraries and Packages


First, we will import the ImageDataGenerator module since we will be leveraging it to train our model in batches.


In [1]:
from keras.preprocessing.image import ImageDataGenerator

In this lab, we will be using the Keras library to build an image classifier, so let's download the Keras library.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Finally, we will be leveraging the ResNet50 model to build our classifier, so let's download it as well.


In [3]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

## Define Global Constants


Here, we will define constants that we will be using throughout the rest of the lab. 

1. We are obviously dealing with two classes, so *num_classes* is 2. 
2. The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3. We will training and validating the model using batches of 100 images.


In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances


In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.


<hr>

## Pretrained Deep Neural Networks

https://www.mathworks.com/help/deeplearning/ug/pretrained-convolutional-neural-networks.html  
https://www.mathworks.com/help/deeplearning/ref/resnet50.html

<img src='img/pretrained_20b.png' width=600px><br>
<img src='img/resnet50.png' width=600px>

You can take a pretrained image classification neural network that has already learned to extract powerful and informative features from natural images and use it as a starting point to learn a new task. The majority of the pretrained neural networks are trained on a subset of the ImageNet database, which is used in the ImageNet Large-Scale Visual Recognition Challenge (ILSVRC). These neural networks have been trained on more than a million images and can classify images into 1000 object categories, such as keyboard, coffee mug, pencil, and many animals. Using a pretrained neural network with transfer learning is typically much faster and easier than training a neural network from scratch.

<hr>

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the *flow_from_directory* method to get the training images as follows:


In [6]:
train_generator = data_generator.flow_from_directory(
    'data/concrete_crack/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30000 images belonging to 2 classes.


**Note**: in this lab, we will be using the full data-set of 40,000 images for training and validation.

Use the *flow_from_directory* method to get the validation images and assign the result to **validation_generator**.


In [7]:
validation_generator = data_generator.flow_from_directory(
    'data/concrete_crack/valid/',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9500 images belonging to 2 classes.


## Build, Compile and Fit Model


In this section, we will start building our model. We will use the Sequential model class from Keras.


In [8]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.


In [9]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.


In [10]:
model.add(Dense(num_classes, activation='softmax'))

You can access the model's layers using the *layers* attribute of our model object. 


In [11]:
model.layers

You can see that our model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.


You can access the ResNet50 layers by running the following:


In [12]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.


In [13]:
model.layers[0].trainable = False

And now using the *summary* attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


Next we compile our model using the **adam** optimizer.


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:


In [16]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.


In [17]:
#%pip instal scipy
import scipy

**Warning**: next code will be execute more than 20 minutes:

In [18]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

C:\Users\usman\AppData\Local\Temp\ipykernel_9920\251737888.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2


300/300 [==============================] - 572s 2s/step - loss: 0.0260 - accuracy: 0.9912 - val_loss: 0.0061 - val_accuracy: 0.9983
Epoch 2/2
300/300 [==============================] - 569s 2s/step - loss: 0.0050 - accuracy: 0.9989 - val_loss: 0.0047 - val_accuracy: 0.9986


Now that the model is trained, you are ready to start using it to classify images.


Since training can take a long time when building deep learning models, it is always a good idea to save your model once the training is complete if you believe you will be using the model again later. You will be using this model in the next module, so go ahead and save your model.


In [19]:
model.save('classifier_resnet_model.h5')

c:\Users\usman\Desktop\122XSUP\week_02\.my_env_02\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Now, you should see the model file *classifier_resnet_model.h5* apprear in the left directory pane.
